# Creating density fields II

We want to compute the density field of a simulation with massive neutrinos in redshift-space. In this case, we need to take into account both the positions of the dark matter and the neutrino particles, together with their velocities and masses.

[![Binder](https://mybinder.org/badge_logo.svg)](https://binder.flatironinstitute.org/v2/user/fvillaescusa/Quijote?filepath=/Tutorials/Density_fields2.ipynb)

In [1]:
import numpy as np
import readgadget
import MAS_library as MASL
import redshift_space_library as RSL

Define the value of the parameters

In [2]:
snapshot = '/home/jovyan/Data/Snapshots/Mnu_p/0/snapdir_003/snap_003' #location of the snapshot
grid     = 256    #the density field will have grid^3 voxels
MAS      = 'CIC'  #Mass-assignment scheme:'NGP', 'CIC', 'TSC', 'PCS'
axis     = 0      #axis along which to move particles to redshift-space (0-X), (1-Y), (2-Z)
verbose  = True   #whether to print information about the progress

Lets read the header and the particle positions and masses (for both dark matter and neutrinos)

In [3]:
# read header
header   = readgadget.header(snapshot)
BoxSize  = header.boxsize/1e3  #Mpc/h
Nall     = header.nall         #Total number of particles
Masses   = header.massarr*1e10 #Masses of the particles in Msun/h
Omega_m  = header.omega_m      #value of Omega_m
Omega_l  = header.omega_l      #value of Omega_l
h        = header.hubble       #value of h
redshift = header.redshift     #redshift of the snapshot
Hubble   = 100.0*np.sqrt(Omega_m*(1.0+redshift)**3+Omega_l)#Value of H(z) in km/s/(Mpc/h)

# read positions and velocities of the particles
pos_c = readgadget.read_block(snapshot, "POS ", [1])/1e3 #positions in Mpc/h
pos_n = readgadget.read_block(snapshot, "POS ", [2])/1e3 #positions in Mpc/h
vel_c = readgadget.read_block(snapshot, "VEL ", [1])     #velocities in km/s
vel_n = readgadget.read_block(snapshot, "VEL ", [2])     #velocities in km/s

print some information about the data read

In [4]:
print('BoxSize = %.3f Mpc/h'%BoxSize)
print('Rdshift = %.3f'%redshift)
print('Mass DM = %.3e'%Masses[1])
print('Mass NU = %.3e'%Masses[2])
print('%.3f < X_DM < %.3f'%(np.min(pos_c[:,0]), np.max(pos_c[:,0])))
print('%.3f < Y_DM < %.3f'%(np.min(pos_c[:,1]), np.max(pos_c[:,1])))
print('%.3f < Z_DM < %.3f'%(np.min(pos_c[:,2]), np.max(pos_c[:,2])))
print('%.3f < X_NU < %.3f'%(np.min(pos_n[:,0]), np.max(pos_n[:,0])))
print('%.3f < Y_NU < %.3f'%(np.min(pos_n[:,1]), np.max(pos_n[:,1])))
print('%.3f < Z_NU < %.3f'%(np.min(pos_n[:,2]), np.max(pos_n[:,2])))

BoxSize = 1000.000 Mpc/h
Rdshift = 0.500
Mass DM = 6.516e+11
Mass NU = 4.930e+09
0.000 < X_DM < 999.992
0.000 < Y_DM < 999.992
0.000 < Z_DM < 999.992
0.000 < X_NU < 999.992
0.000 < Y_NU < 999.992
0.000 < Z_NU < 999.992


Now lets move particles to redshift space along the X-axis

In [5]:
# move dark matter particles to redshift-space
RSL.pos_redshift_space(pos_c, vel_c, BoxSize, Hubble, redshift, axis)

# move neutrino particles to redshift-space
RSL.pos_redshift_space(pos_n, vel_n, BoxSize, Hubble, redshift, axis)

Now construct the density field of matter (DM+NU) 

In [6]:
# define the matrix holding the density field
delta = np.zeros((grid,grid,grid), dtype=np.float32)

# define two arrays with the masses of the DM and NU particles
mass_c = np.ones(pos_c.shape[0], dtype=np.float32)*Masses[1]
mass_n = np.ones(pos_n.shape[0], dtype=np.float32)*Masses[2]

# construct the density field
MASL.MA(pos_c, delta, BoxSize, MAS, W=mass_c, verbose=verbose)
MASL.MA(pos_n, delta, BoxSize, MAS, W=mass_n, verbose=verbose)


Using CIC mass assignment scheme with weights
Time taken = 5.397 seconds


Using CIC mass assignment scheme with weights
Time taken = 23.766 seconds



Make some checks 

In [7]:
# check the total mass in the density field
Mtot1 = np.sum(delta, dtype=np.float64)
Mtot2 = np.sum(mass_c, dtype=np.float64) + np.sum(mass_n, dtype=np.float64)
print('%.3e should be equal to\n%.3e'%(Mtot1,Mtot2))

8.812e+19 should be equal to
8.812e+19


If needed, the overdensity field can be easily computed

In [8]:
delta /= np.mean(delta, dtype=np.float64);  delta -= 1.0
print('%.3f < delta < %.3f'%(np.min(delta), np.max(delta)))
print('<delta> = %.3f'%np.mean(delta))

-0.994 < delta < 34.707
<delta> = 0.000
